In [1]:
from OthelloEnv import OthelloEnv

import gym
import torch as th
import torch.nn as nn

from stable_baselines3 import A2C, PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_checker import check_env

In [2]:
env = OthelloEnv()
check_env(env)

/Users/jacobswe/Python-Projects/Othello-AI/venv/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:26: UserWarning: It seems that your observation  is an image but the `dtype` of your observation_space is not `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(
/Users/jacobswe/Python-Projects/Othello-AI/venv/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:34: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
/Users/jacobswe/Python-Projects/Othello-AI/venv/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:47: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom feature extractor cf. htt

In [3]:
model = A2C('MlpPolicy', env, verbose=0, tensorboard_log='logs')
model.learn(total_timesteps=100000)

In [4]:
class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=(4, 4), stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding='same'),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256),
)
model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=0, tensorboard_log='logs')
model.learn(100000)

/Users/jacobswe/Python-Projects/Othello-AI/venv/lib/python3.9/site-packages/torch/nn/modules/conv.py:442: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  return F.conv2d(input, weight, bias, self.stride,
